In [1]:
#!/usr/bin/env python
# -------------------------------------------------------------------------

from matplotlib import colors
import datetime
import os
from osgeo import gdal
from alborex_functions import *
%matplotlib inline

In [2]:
dd = datetime.datetime.now()
figdate = str(dd.year) + str(dd.month) + str(dd.day)
figtitle = dd.strftime("%d %B %Y")
timemin = (dd - datetime.datetime(1970,1,1)).total_seconds() - 15. * 86400

# Files and directories

In [3]:
figname = "Turtle_salinity_current2_" + figdate
visiblefile = "/data_local/Satellite/Visible/nasa-worldview-2016-10-16.tiff"
romsfile = "http://thredds.socib.es/thredds/dodsC/operational_models/oceanographical/hydrodynamics/wmop/latest.nc"
turtledir = "http://thredds.priv.socib.es/thredds/dodsC/animal/turtle/"
turtlelist = ['turtle_earl-scb_splash005/L1/2016/07/dep0001_turtle-earl_scb-splash005_L1_2016-07-26.nc',
              'turtle_eddy-scb_splash003/L1/2016/07/dep0001_turtle-eddy_scb-splash003_L1_2016-07-13.nc',
              'turtle_chloe-scb_splash002/L1/2016/07/dep0001_turtle-chloe_scb-splash002_L1_2016-07-20.nc',
              'turtle_pixel-scb_splash001/L1/2016/06/dep0001_turtle-pixel_scb-splash001_L1_2016-06-22.nc']
turtlelogo = "/home/ctroupin/Presentations/figures4presentations/logo/turtle.png"
figdir = '/home/ctroupin/Pictures/SOCIB/'

# Configuration and options

In [14]:
projection = False

# Regions of interest and box for the experiment
dlon, dlat = 2.0, 1.0
coordinates = (-6.0, 9.2001, 35, 44)
res = 'i'
valex = -999.

variable1 = 'sea_surface_temperature'
variable2 = 'sea_surface_salinity'
cmap = plt.cm.RdYlBu_r

vmin, vmax, dvar = 36.5, 38.5, 0.5
normpsal = colors.Normalize(vmin=vmin, vmax=vmax)
normvel = colors.Normalize(vmin=0.0, vmax=0.5)
bounds = np.arange(vmin, vmax + .0001, dvar)

# Compute min and max values 
sstmin, sstmax = 16., 21.
normsst = colors.Normalize(vmin=sstmin, vmax=sstmax)
boundsst = np.arange(sstmin, sstmax + .001, 1.0)

if projection:
    fig, m, ax = prepare_map(coordinates, res)
    plt.close()

# Load data 

In [10]:
def get_turtle_coords(turtlefile, timemin):
    with netcdf.Dataset(turtlefile) as nc:
        timeturtle = nc.variables['time'][:]
        goodtime = np.where(timeturtle > timemin)[0]
        lonturtleQC = nc.variables['QC_LON'][goodtime]
        latturtleQC = nc.variables['QC_LAT'][goodtime]
        goodcoord = np.where( (lonturtleQC == 1 ) & (latturtleQC == 1))[0]
        lonturtle = nc.variables['LON'][goodtime][goodcoord]
        latturtle = nc.variables['LAT'][goodtime][goodcoord]
    return lonturtle, latturtle

## Read visible image

In [11]:
gtif = gdal.Open(visiblefile)
gtif.GetProjectionRef()

#Set the plot axis limits to the proper map coordinates.
arr = gtif.ReadAsArray()
trans = gtif.GetGeoTransform()
extent = (trans[0], trans[0] + gtif.RasterXSize*trans[1],
          trans[3] + gtif.RasterYSize*trans[5], trans[3])

In [12]:
# Load model salinity and temperature
with netcdf.Dataset(romsfile) as nc:
    latts = nc.variables['lat_rho'][:]
    lonts = nc.variables['lon_rho'][:]
    goodlat = np.where( (latts >= coordinates[2]) & (latts <= coordinates[3]))[0]
    goodlon = np.where( (lonts >= coordinates[0]) & (lonts <= coordinates[1]))[0]
    salinity = np.mean(nc.variables['salt'][:, goodlat, goodlon], axis=0)
    temperature = np.mean(nc.variables['temp'][:, goodlat, goodlon], axis=0)
    latts = latts[goodlat]
    lonts = lonts[goodlon]
    latuv = nc.variables['lat_uv'][:]
    lonuv = nc.variables['lon_uv'][:]
    goodlat = np.where( (latuv >= coordinates[2]) & (latuv <= coordinates[3]))[0]
    goodlon = np.where( (lonuv >= coordinates[0]) & (lonuv <= coordinates[1]))[0]
    u = np.mean(nc.variables['u'][:, goodlat, goodlon], axis=0)
    v = np.mean(nc.variables['v'][:, goodlat, goodlon], axis=0)
    latuv = latuv[goodlat]
    lonuv = lonuv[goodlon]
if projection:
    lonts, latts = np.meshgrid(lonts, latts)
    lonuv, latuv = np.meshgrid(lonuv, latuv)
    lonts, latts = m(lonts, latts)
    lonuv, latuv = m(lonuv, latuv)

# Make the plot

In [15]:
#fig, m, ax = prepare_map(coordinates, res)

fig = plt.figure()

# Load turtle data
for turtle in turtlelist[1:]:
    turtlefile = turtledir + turtle
    lonturtle, latturtle = get_turtle_coords(turtlefile, timemin)
    
    if projection:
        lonturtle, latturtle = m(lonturtle, latturtle)

    plt.plot(lonturtle, latturtle, 'ko-', zorder=5, markersize=0.2, linewidth=0.5)
    plt.plot(lonturtle[-1], latturtle[-1], 'wo-', zorder=5, markersize=3, linewidth=0.5)

N = 5
speed = np.sqrt(u * u + v * v)
speed = np.ma.masked_greater(speed, 1.5)

plt.streamplot(lonuv, latuv, u, v, color=speed,
                arrowsize=1.5, norm=normvel,cmap=plt.cm.gray_r,
                density=10, linewidth=.25, zorder=5)

pcm = plt.pcolormesh(lonts, latts, salinity, norm=normpsal, cmap=cmap, zorder=4)

'''
cbar = plt.colorbar(pcm, cmap=cmap, orientation='vertical', pad=0.05, aspect=15,
                        shrink=0.75, extend='both')
cbar.set_ticks(bounds)
'''
# x and y ticks (no line plotted for better visibility)
'''
m.drawparallels(np.arange(round(coordinates[2]), coordinates[3], dlat), linewidth=0.,
               labels=[1, 0, 0, 0], fontname='Times New Roman', fontsize=16, zorder=1)
m.drawmeridians(np.arange(round(coordinates[0]), coordinates[1], dlon), linewidth=0.,
                labels=[0, 0, 0, 1], fontname='Times New Roman', fontsize=16, zorder=1)
'''
# m.drawmapscale(-0.45, 35.1, -0.25, 35.1, 100, barstyle='simple', units='km', fontsize=12, zorder=3)

plt.imshow(arr[:3,:,:].transpose((1, 2, 0)), extent=extent, zorder=3, alpha=1)

# Coastline and continent
#m.fillcontinents(ax=ax, color='0.9', zorder=2)
#m.drawcoastlines(ax=ax, linewidth=0.1, color='k', zorder=4)
# plt.title(figtitle, fontsize=20)

plt.xlim(coordinates[0], coordinates[1])
plt.ylim(coordinates[2], coordinates[3])
plt.axis('off')
plt.savefig(os.path.join(figdir, figname + 'c'), dpi=300, facecolor='w', edgecolor='w',
            transparent=False, bbox_inches='tight', pad_inches=0.1)

# plt.show()
plt.close()